In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import torch.nn as nn
import torch
import random as rd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
from statsmodels.distributions.empirical_distribution import StepFunction

In [19]:
def create_dataset(original_data, input_list, output):
    selected_columns = input_list + output
    df = original_data[selected_columns]
    df = df.dropna()
    return df

def get_zscored(df):
    df_raw = df.loc[:,~df.columns.str.contains('percentile')]
    df_raw = df_raw.loc[:,df_raw.columns.isin(labs.keys())]
    #df_raw.columns = df_raw.columns.map(labs)
    drop = ['50947', '50934', '51678', '52135', '51133', '52069', '52074', '52073', '52075']
    df_common = df_raw.drop(drop, axis=1)
    df_z_scores = (df_common - df_common.mean()) / df_common.std()
    return df_z_scores

def get_percentile(df):
    df_raw = df.loc[:,df.columns.str.contains('percentile')]
    drop = ['50947', '50934', '51678', '52135', '51133', '52069', '52074', '52073', '52075']
    drop = [x+"_percentile" for x in drop]
    df_common = df_raw.drop(drop, axis=1)
    df_common.columns = [x.split("_")[0] for x in df_common.columns]
    return df_common

def get_raw(df):
    df_raw = df.loc[:,~df.columns.str.contains('percentile')]
    df_raw = df_raw.loc[:,df_raw.columns.isin(labs.keys())]
    #df_raw.columns = df_raw.columns.map(labs)
    drop = ['50947', '50934', '51678', '52135', '51133', '52069', '52074', '52073', '52075']
    df_common = df_raw.drop(drop, axis=1)
    return df_common

In [30]:
class NaNAwareECDF(StepFunction):
    """
    Very similar to the statsmodels ECDF class
    (https://www.statsmodels.org/stable/generated/statsmodels.distributions.empirical_distribution.ECDF.html)
    except that it computes a NaN-aware ECDF by filling values corresponding to np.nan with np.nan.

    Source: https://stackoverflow.com/a/68959320
    """
    def __init__(self, x, side='right'):
        x = np.sort(x)

        # count number of non-nan's instead of length
        nobs = np.count_nonzero(~np.isnan(x))

        # fill the y values corresponding to np.nan with np.nan
        y = np.full_like(x, np.nan)
        y[:nobs]  = np.linspace(1./nobs,1,nobs)
        super(NaNAwareECDF, self).__init__(x, y, side=side, sorted=True)

    def inverse_ecdf(self, p):
        """
        Inverse ECDF (quantile function) method.
        Maps a percentile `p` (between 0 and 1) to the corresponding value on the ECDF curve.

        Parameters:
        -----------
        p : float or array-like
            Percentile(s) to map to the corresponding value(s).

        Returns:
        --------
        float or array-like
            The value(s) corresponding to the given percentile(s).
        """
        if np.any(p < 0) or np.any(p > 1):
            raise ValueError("Percentile `p` must be between 0 and 1.")

        # Get the sorted non-NaN values and their corresponding ECDF values
        x = self.x[~np.isnan(self.y)]
        y = self.y[~np.isnan(self.y)]

        # Use interpolation to find the value corresponding to the percentile `p`
        return np.interp(p, y, x)

In [31]:
labs = {
    "51221": "Hematocrit",
    "51265": "Platelet Count",
    "50912": "Creatinine",
    "50971": "Potassium",
    "51222": "Hemoglobin",
    "51301": "White Blood Cells",
    "51249": "MCHC",
    "51279": "Red Blood Cells",
    "51250": "MCV",
    "51248": "MCH",
    "51277": "RDW",
    "51006": "Urea Nitrogen",
    "50983": "Sodium",
    "50902": "Chloride",
    "50882": "Bicarbonate",
    "50868": "Anion Gap",
    "50931": "Glucose",
    "50960": "Magnesium",
    "50893": "Calcium, Total",
    "50970": "Phosphate",
    "51237": "INR(PT)",
    "51274": "PT",
    "51275": "PTT",
    "51146": "Basophils",
    "51256": "Neutrophils",
    "51254": "Monocytes",
    "51200": "Eosinophils",
    "51244": "Lymphocytes",
    "52172": "RDW-SD",
    "50934": "H",
    "51678": "L",
    "50947": "I",
    "50861": "Alanine Aminotransferase (ALT)",
    "50878": "Asparate Aminotransferase (AST)",
    "50813": "Lactate",
    "50863": "Alkaline Phosphatase",
    "50885": "Bilirubin, Total",
    "50820": "pH",
    "50862": "Albumin",
    "50802": "Base Excess",
    "50821": "pO2",
    "50804": "Calculated Total CO2",
    "50818": "pCO2",
    "52075": "Absolute Neutrophil Count",
    "52073": "Absolute Eosinophil Count",
    "52074": "Absolute Monocyte Count",
    "52069": "Absolute Basophil Count",
    "51133": "Absolute Lymphocyte Count",
    "50910": "Creatine Kinase (CK)",
    "52135": "Immature Granulocytes"
}
labs_reversed = {value: key for key, value in labs.items()}

In [46]:
df_train = pd.read_csv(r"C:\Users\joshu\Downloads\train.csv")
df_test = pd.read_csv(r"C:\Users\joshu\Downloads\test (1).csv")
df_val = pd.read_csv(r"C:\Users\joshu\Downloads\val.csv")

df_z_scores = get_zscored(df_train)
df_percentiles = get_percentile(df_train)
df_test_z = get_zscored(df_test)
df_test_raw = get_raw(df_test)
df_test_percentile = get_percentile(df_test)
df_full = pd.concat([df_train, df_test, df_val])

In [47]:
var_cdfs = {}
for var in labs.keys():
    ecdf = NaNAwareECDF(df_train[var])
    var_cdfs[var] = ecdf

In [48]:
#df_z_scores.to_excel("full_train_z_scored.xlsx")

In [49]:
encode = lambda x: [labs_reversed[i] for i in x]
decode = lambda x: [labs[i] for i in x]

In [50]:
def rf2(xs, y, n_estimators=200, max_samples=0.8,
       max_features=0.8206143222402672, min_samples_leaf=1, max_depth= 13, **kwargs):
    return RandomForestRegressor(n_jobs=-1, max_depth=max_depth, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)
MSE = nn.MSELoss()

In [51]:
total_feats = ['Platelet Count',
 'Basophils',
 'Sodium',
 'Urea Nitrogen',
 'PTT',
 'Albumin',
 'Neutrophils',
 'pH',
 'pO2',
 'Bicarbonate',
 'Hematocrit',
 'Potassium']

In [52]:
cols = decode(df_z_scores.columns.to_list())
targets = list(set(cols) - set(total_feats))
targets

['Red Blood Cells',
 'MCH',
 'Chloride',
 'RDW',
 'MCV',
 'Lymphocytes',
 'Hemoglobin',
 'Creatine Kinase (CK)',
 'Anion Gap',
 'Glucose',
 'Calculated Total CO2',
 'Monocytes',
 'MCHC',
 'Bilirubin, Total',
 'Creatinine',
 'Lactate',
 'Alkaline Phosphatase',
 'INR(PT)',
 'Magnesium',
 'Phosphate',
 'Calcium, Total',
 'Alanine Aminotransferase (ALT)',
 'Base Excess',
 'PT',
 'Eosinophils',
 'Asparate Aminotransferase (AST)',
 'White Blood Cells',
 'pCO2',
 'RDW-SD']

In [57]:
for target in targets:
    
    ecdf =  var_cdfs[encode([target])[0]]
    
    inputs = encode(total_feats)
    
    output = encode([target])
    
    df = create_dataset(df_percentiles, inputs, output)
    
    df_test = create_dataset(df_test_raw, inputs, output)
    
    x_df = df[encode(total_feats)]
    y_df = df[encode([target])]
    
    x_test_df = df_test[encode(total_feats)]
    y_test_df = df_test[encode([target])]

    y = y_df.to_numpy()
    x = x_df.to_numpy()
    
    x_test = x_test_df.to_numpy()
    y_test = y_test_df.to_numpy()
    
    y = y.ravel()
    y_test = y_test.ravel() # prevents mismatched shapes in the second dimension

    m = rf2(x,y)

    y_preds = m.predict(x_test)
    y_preds_ecdf = ecdf.inverse_ecdf(y_preds)

    loss = float(MSE(torch.tensor(y_test), torch.tensor(y_preds_ecdf)))

    #loss_df.loc[len(loss_df)] = [target, loss]
    print(target)
    print(loss)

Red Blood Cells
1.7975755455970672
MCH
6.824141132776231
Chloride
47.74744661095636
RDW
6.059350046425255
MCV
57.341689879294336
Lymphocytes
186.96702878365832
Hemoglobin
16.194493964716802
Creatine Kinase (CK)
219086334.10286254
Anion Gap
38.404828226555246
Glucose
10656.474466109563
Calculated Total CO2
41.32033426183844
Monocytes
20.191652739090067
MCHC
2.732293407613743
Bilirubin, Total
2.0024879923150816
Creatinine
7.904029712163417
Lactate
5.799388349514563
Alkaline Phosphatase
27376.52727272727
INR(PT)
2.142246982358403
Magnesium
0.24577251184834137
Phosphate
2.893661835748792
Calcium, Total
1.2028119001919382
Alanine Aminotransferase (ALT)
325067.06889952155
Base Excess
39.13091922005571
PT
253.32573816155985
Eosinophils
2.5852367688022286
Asparate Aminotransferase (AST)
771220.5971291866
White Blood Cells
70.75997214484678
pCO2
161.2534818941504
RDW-SD
76.53601591836741


In [89]:
target = 'Eosinophils'

ecdf =  var_cdfs[encode([target])[0]]
    
inputs = encode(total_feats)

output = encode([target])

df = create_dataset(df_percentiles, inputs, output)

df_test = create_dataset(df_test_raw, inputs, output)
df_test_percentiles = create_dataset(df_test_percentile, inputs, output)

x_df = df[encode(total_feats)]
y_df = df[encode([target])]

x_test_df = df_test[encode(total_feats)]
y_test_df = df_test[encode([target])]
y_test_percentiles =df_test_percentile[encode([target])]

y = y_df.to_numpy()
x = x_df.to_numpy()

x_test = x_test_df.to_numpy()
y_test = y_test_df.to_numpy()

y = y.ravel()
y_test = y_test.ravel() # prevents mismatched shapes in the second dimension

m = rf2(x,y)

y_preds = m.predict(x_test)
y_preds_ecdf = ecdf.inverse_ecdf(y_preds)

loss = float(MSE(torch.tensor(y_test), torch.tensor(y_preds_ecdf)))

preds_df = pd.DataFrame(y_preds_ecdf)

In [90]:
print(preds_df.min())
print(preds_df.mean())
print(preds_df.max())

0    0.7
dtype: float64
0    0.70585
dtype: float64
0    0.8
dtype: float64


In [91]:
print(y_test_df.min())
print(y_test_df.mean())
print(y_test_df.max())

51200    0.0
dtype: float64
51200    1.042804
dtype: float64
51200    7.0
dtype: float64


In [92]:
print(y_preds.min())
print(y_preds.mean())
print(y_preds.max())

0.4678326239457708
0.4795667402648126
0.5123348703397174


In [93]:
print(y_test_percentiles.min())
print(y_test_percentiles.mean())
print(y_test_percentiles.max())

51200    0.164814
dtype: float64
51200    0.512446
dtype: float64
51200    0.999054
dtype: float64
